# II. Spatial Autocorrelation


> **РАЗДЕЛ В РАЗРАБОТКЕ**
> Материалы будут дополнены в ближайшее время


## Пространственная автокорреляция

**Пространственная автокорреляция** — это статистический метод, который используется для измерения степени зависимости или сходства значений переменных в пространственно распределенных объектах. Пространственная автокорреляция исследует, как значения переменной в одном месте соотносятся с значениями в соседних местах.

Если объекты, расположенные рядом друг с другом, имеют схожие значения, то это свидетельствует о высоком уровне пространственной автокорреляции. Напротив, если соседние объекты сильно различаются, то автокорреляция будет низкой.

### Виды пространственной автокорреляции:

1. **Положительная автокорреляция** — когда объекты, расположенные близко друг к другу, имеют схожие значения.
2. **Отрицательная автокорреляция** — когда объекты, расположенные рядом, имеют противоположные значения.
3. **Нулевая автокорреляция** — когда нет зависимости между значениями в соседних точках.


## 0. Подготовка данных


Импортируем библиотеки


In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np

from shapely.geometry import box

import matplotlib.pyplot as plt

from pysal.explore import esda

**Загружаем данные о плотности населения Краснодара, агрегированные по квадратной сетке регулярных ячеек**


In [ ]:
sq_grid = gpd.read_file('./data/krs_pop_sqgrid.geojson')

sq_grid.explore(tiles='cartodbpositron')

## 1. Global Moran I


**Индекс Морана** — это один из наиболее часто используемых методов для оценки пространственной автокорреляции. Этот индекс измеряет, насколько значения переменной в соседних местах схожи. Он определяется по формуле:

```{math}
I = \frac{N}{W} \cdot
\frac{\sum_i \sum_j w_{ij}(x_i - \overline{x})(x_j - \overline{x})}
{\sum_i (x_i - \overline{x})^2}
```

где:

- ( N ) — количество наблюдений,
- ( W ) — сумма всех весов в матрице соседства,
- ( w\_{ij} ) — вес (или мера близости) между объектами ( i ) и ( j ),
- ( x_i ) и ( x_j ) — значения переменной в точках ( i ) и ( j ),
- ( \overline{x} ) — среднее значение переменной.

Значения индекса Морана интерпретируются следующим образом:

- ( I > 0 ) — положительная пространственная автокорреляция (соседние объекты схожи)
- ( I < 0 ) — отрицательная пространственная автокорреляция (соседние объекты сильно различаются)
- ( I = 0 ) — отсутствие пространственной автокорреляции


#### Рассчитаем Global Moran I для значений плотности населения в Краснодаре


Посмотрим матрицу пространственных весов


In [ ]:
from libpysal.weights import Queen
w = Queen.from_dataframe(sq_grid, ids=sq_grid.index.tolist())

In [ ]:
# Рассчитываем индекс Морана для плотности населения
# grid['pop_density'] - значения плотности в каждой ячейке
# w - матрица весов
moran_pop = esda.Moran(sq_grid['pop_density'], w)

In [ ]:
print(f"Moran's I для кол-ва ДТП: {moran_pop.I}")
print(f"p-value для кол-ва ДТП: {moran_pop.p_sim}")

- значение индекса - высокое положительное значение, которое говорит о сильной пространственной автокорреляции.
- **p = 0.001** -- высокая статистическая значимость.


Важно исправить и убрать из данных NaN


## 2. Local Morans's I


#### Рассчитаем Local Moran's I для значений плотности населения в Краснодаре


**Local Moran’s I (LISA — Local Indicators of Spatial Association)** — это показатель локальной пространственной автокорреляции.
Он показывает **не общую (глобальную)** зависимость, а то, **как каждая отдельная ячейка связана со своими соседями**.

Local Moran помогает обнаружить:

- **локальные кластеры**:

  - **HH** — высокая величина окружена высокими
  - **LL** — низкая величина окружена низкими

- **пространственные аномалии**:

  - **HL** — высокая величина среди низких
  - **LH** — низкая величина среди высоких

**Формула Local Moran’s I**

```{math}
I_i = z_i \sum_{j} w_{ij} z_j
```

Где:

- ( I_i ) — локальный индекс Морана для объекта ( i ),
- ( z_i = \frac{x_i - \bar{x}}{s} ) — стандартизированное значение переменной,
- ( w\_{ij} ) — вес из матрицы соседства (1, если ( j ) — сосед; 0 — иначе),
- сумма берётся по всем соседям ( j ),
- ( z_j ) — стандартизированные значения соседей.

**Как интерпретировать Local Moran I**

- ( I_i > 0 )
  объект похож на своих соседей (кластер HH или LL)

- ( I_i < 0 )
  объект сильно отличается от соседей (аномалия HL или LH)


In [ ]:
# Рассчитываем локальный индекс Морана для ДТП
local_moran = esda.Moran_Local(sq_grid['pop_density'], w)

# Получаем локальные значения индекса Морана для каждого региона
local_moran_values = local_moran.Is  # Это значения индекса Морана для каждого элемента
print(local_moran_values)


#### Построим LISA Cluster Map


В local_moran.q мы можем узнать к каому кластеру (HH, LL, HL, LH) попадает каждая из ячеек


In [ ]:
local_moran.q

**LISA Cluster Map** — это карта локальных кластеров индекса Морана, которая показывает, где именно в пространстве формируются значимые группы высоких или низких значений. В отличие от глобального индекса Морана, который оценивает пространственную зависимость в целом, LISA анализируется отдельно для каждой ячейки и её соседей.

Карта выделяет четыре типа зон: **HH** (высокие значения среди высоких), **LL** (низкие среди низких), **HL** (высокие среди низких — аномальные пики) и **LH** (низкие среди высоких )


In [ ]:
# категории LISA в правильном порядке
cluster_labels = np.array(['Not significant', 'HH', 'LH', 'LL', 'HL'])
labels = cluster_labels[local_moran.q]

sq_grid['lisa_cluster'] = labels

# Цвета для категорий
colors = {
    'HH': 'red',
    'LL': 'blue',
    'LH': 'lightblue',
    'HL': 'pink',
    'Not significant': 'lightgrey'
}

sq_grid.explore(
    column='lisa_cluster',
    categories=list(colors.keys()),   # список категорий
    color=[colors[c] for c in colors], # список цветов в том же порядке
    categorical=True,
    legend=True,
    tiles='cartodbpositron'
)
